In [1]:
import pyspark
import os
from pyspark.sql import SparkSession
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder \
    .appName("Medallion Architecture") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/22 12:14:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
raw_path = os.path.join("..","data","*.csv")

df = spark.read.csv(raw_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- YEAR: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- EDUC: integer (nullable = true)
 |-- ETHNIC: integer (nullable = true)
 |-- RACE: integer (nullable = true)
 |-- GENDER: integer (nullable = true)
 |-- SPHSERVICE: integer (nullable = true)
 |-- CMPSERVICE: integer (nullable = true)
 |-- OPISERVICE: integer (nullable = true)
 |-- RTCSERVICE: integer (nullable = true)
 |-- IJSSERVICE: integer (nullable = true)
 |-- MH1: integer (nullable = true)
 |-- MH2: integer (nullable = true)
 |-- MH3: integer (nullable = true)
 |-- SUB: integer (nullable = true)
 |-- MARSTAT: integer (nullable = true)
 |-- SMISED: integer (nullable = true)
 |-- SAP: integer (nullable = true)
 |-- EMPLOY: integer (nullable = true)
 |-- DETNLF: integer (nullable = true)
 |-- VETERAN: integer (nullable = true)
 |-- LIVARAG: integer (nullable = true)
 |-- NUMMHS: integer (nullable = true)
 |-- TRAUSTREFLG: integer (nullable = true)
 |-- ANXIETYFLG: integer (nullable = true)
 |-- ADHDFLG

24/05/22 12:14:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+---+----+------+----+------+----------+----------+----------+----------+----------+---+---+---+---+-------+------+---+------+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+--------+------+-----------+
|YEAR|AGE|EDUC|ETHNIC|RACE|GENDER|SPHSERVICE|CMPSERVICE|OPISERVICE|RTCSERVICE|IJSSERVICE|MH1|MH2|MH3|SUB|MARSTAT|SMISED|SAP|EMPLOY|DETNLF|VETERAN|LIVARAG|NUMMHS|TRAUSTREFLG|ANXIETYFLG|ADHDFLG|CONDUCTFLG|DELIRDEMFLG|BIPOLARFLG|DEPRESSFLG|ODDFLG|PDDFLG|PERSONFLG|SCHIZOFLG|ALCSUBFLG|OTHERDISFLG|STATEFIP|DIVISION|REGION|     CASEID|
+----+---+----+------+----+------+----------+----------+----------+----------+----------+---+---+---+---+-------+------+---+------+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+--------+------+-----------+
|2021| 

In [3]:
bronze_path = os.path.join("..","data","medallion_layers","bronze")
df.write.partitionBy("REGION", "STATEFIP").mode("overwrite").parquet(bronze_path)



24/05/22 12:14:42 WARN MemoryManager: Total allocation exceeds 95.00% (1,013,579,763 bytes) of heap memory
Scaling row group sizes to 94.40% for 8 writers
24/05/22 12:14:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,013,579,763 bytes) of heap memory
Scaling row group sizes to 94.40% for 8 writers
24/05/22 12:14:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,013,579,763 bytes) of heap memory
Scaling row group sizes to 94.40% for 8 writers
24/05/22 12:14:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,013,579,763 bytes) of heap memory
Scaling row group sizes to 94.40% for 8 writers
24/05/22 12:14:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,013,579,763 bytes) of heap memory
Scaling row group sizes to 94.40% for 8 writers
24/05/22 12:14:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,013,579,763 bytes) of heap memory
Scaling row group sizes to 94.40% for 8 writers
24/05/22 12:14:46 WARN MemoryManager: Total allocation exceeds 95.00% 

In [4]:
bronze_cleaned_path = os.path.join("..","data","medallion_layers","bronze_cleaned")
bronze_df = spark.read.parquet(bronze_path, header=True, inferSchema=True)
cleaned_bronze_df = bronze_df.na.drop()
cleaned_bronze_df.show(5)
cleaned_bronze_df.write.partitionBy("REGION", "STATEFIP").mode("overwrite").parquet(bronze_cleaned_path)

+----+---+----+------+----+------+----------+----------+----------+----------+----------+---+---+---+---+-------+------+---+------+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+-----------+------+--------+
|YEAR|AGE|EDUC|ETHNIC|RACE|GENDER|SPHSERVICE|CMPSERVICE|OPISERVICE|RTCSERVICE|IJSSERVICE|MH1|MH2|MH3|SUB|MARSTAT|SMISED|SAP|EMPLOY|DETNLF|VETERAN|LIVARAG|NUMMHS|TRAUSTREFLG|ANXIETYFLG|ADHDFLG|CONDUCTFLG|DELIRDEMFLG|BIPOLARFLG|DEPRESSFLG|ODDFLG|PDDFLG|PERSONFLG|SCHIZOFLG|ALCSUBFLG|OTHERDISFLG|DIVISION|     CASEID|REGION|STATEFIP|
+----+---+----+------+----+------+----------+----------+----------+----------+----------+---+---+---+---+-------+------+---+------+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+-----------+------+--------+
|2021| 